In [42]:
import mysql.connector
import pandas as pd

def show(db, query):
    con = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        passwd = 'Milkyboba321:)',
        database = db
    )

    executor = con.cursor()
    executor.execute(query)

    table = executor.fetchall()

    con.close()
    return pd.DataFrame(data=table, columns=executor.column_names)

## Problem Statement

---

X Clothing Company prides themselves on providing an optimised range of clothing and lifestyle wear for the modern adventurer! The CEO has asked you to assist the team’s merchandising teams analyse their sales performance and generate a basic financial report to share with the wider business.

## Avaliable Data

---

For this case study there is a total of 2 datasets, product details and product sales.

### T**able 1: Product Details**

Contains all information about the entire range that company sells in their store.

### **Table 2: Product Sales**

Contains product level information for all the transactions made for company including quantity, price, percentage discount, member status, a transaction ID and also the transaction timestamp.

## Case Study Questions

---

The following questions can be considered key business questions and metrics that our team requires for their monthly reports. Each question can be answered using a single query, but as you are writing the SQL to solve each individual problem, keep in mind how you would generate all of these metrics in a single SQL script which our team can run each month.

### Question 1: **Sales Analysis**

1. What was the total quantity sold for all products?

In [58]:
q = '''SELECT sum(qty) as total FROM product_sales '''

show(db='clothing_company', query=q)

,total
0,45216


2. What is the total generated revenue for all products before discounts?

In [45]:
q = '''SELECT sum(qty*price) as revenue FROM product_sales '''

show(db='clothing_company', query=q)

,revenue
0,1289453


3. What was the total discount amount for all products?

In [46]:
q = '''SELECT sum((qty*price*discount)/100) as total_discount FROM product_sales '''

show(db='clothing_company', query=q)

,total_discount
0,156229.1400


## Question 2: Transaction Analysis

1. How many unique transactions were there?

In [47]:
q = '''SELECT count(distinct(txn_id)) as unique_trans FROM product_sales '''

show(db='clothing_company', query=q)

,unique_trans
0,2500


2. What is the average unique products purchased in each transaction?

In [48]:
q = '''SELECT round(count(prod_id)/count(distinct(txn_id)),0) as avg_unique FROM product_sales'''

show(db='clothing_company', query=q)

,avg_unique
0,6


3. What are the 25th, 50th and 75th percentile values for the revenue per transaction?

In [49]:
q = ''' 
WITH revenue as(SELECT txn_id, 
    sum((qty*price)-((qty*price*discount)/100)) AS rev,
    NTILE(100) OVER(ORDER BY sum((qty*price)-((qty*price*discount)/100))) AS percentile 
    FROM product_sales
    GROUP BY txn_id
)
SELECT
    max(rev) as Revenue, percentile
FROM
    revenue
WHERE
    percentile IN (25, 50, 75)
GROUP BY
    percentile;
'''
show(db='clothing_company', query=q)

,Revenue,percentile
0,326.1800,25
1,441.0000,50
2,572.7500,75


4. What is the average discount value per transaction?

In [68]:
q = ''' 
WITH avg_disc as(
    SELECT txn_id, 
    sum((qty*price*discount)/100)as disc
    from product_sales
    GROUP BY txn_id)

SELECT round(avg(disc),2) as Avg_Disc
FROM avg_disc
'''

show(db='clothing_company', query=q)

,Avg_Disc
0,62.49


5. What is the percentage split of all transactions for members vs non-members?

In [69]:
q='''SELECT member as Member, 
round(count(distinct(txn_id))/(SELECT count(distinct(txn_id))FROM product_sales)*100,2) as Percentage
FROM product_sales
GROUP BY member;'''

show(db='clothing_company', query=q)

,Member,Percentage
0,f,39.80
1,t,60.20


6. What is the average revenue for member transactions and non-member transactions?

In [52]:
q='''SELECT member as Member, 
avg((qty*price)-(qty*price*discount)/100) as average
FROM product_sales
GROUP BY member;'''

show(db='clothing_company', query=q)

,Member,average
0,t,75.43054078
1,f,74.53558668


## Question 3: Product Analysis

1. What is the percentage split of total revenue by category?

In [53]:
q= '''WITH table_join as(SELECT * FROM product_sales s
	LEFT JOIN product_details d
    ON s.prod_id = d.product_id)
SELECT
    category_name as Category,
    round(sum((qty*price)-(qty*price*discount)/100)/
    (SELECT sum((qty*price)-(qty*price*discount)/100) from table_join)*100,2) as Percentage
FROM table_join
GROUP BY category_name;'''

show(db='clothing_company', query=q)

,Category,Percentage
0,Womens,44.63
1,Mens,55.37


2. What is the total transaction “penetration” for each product? (hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)

In [54]:
q='''WITH tab_prod as(SELECT prod_id, count(distinct(txn_id)) as total_prod 
	    FROM product_sales s 
	    LEFT JOIN product_details d
	    ON s.prod_id = d.product_id
	    GROUP BY prod_id),
	total_trans as(SELECT count(distinct(txn_id)) as total_trans 
        FROM product_sales)
SELECT prod_id, total_prod/total_trans as penetration 
    FROM tab_prod
	CROSS JOIN total_trans;'''


show(db='clothing_company', query=q)

,prod_id,penetration
0,2a2353,0.5072
1,2feb6b,0.5032
2,5d267b,0.5072
3,72f5d4,0.5000
4,9ec847,0.5100
5,b9a74d,0.4972
6,c4a632,0.5096
7,c8d436,0.4968
8,d5e9a6,0.4988
9,e31d39,0.4972


3. What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?

In [55]:
q= '''  SELECT ps1.prod_id AS product_1, ps2.prod_id AS product_2, ps3.prod_id AS product_3, COUNT(*) AS trans_count
        FROM product_sales ps1
        JOIN product_sales ps2 USING (txn_id)
        JOIN product_sales ps3 USING (txn_id)
        WHERE ps1.prod_id != ps2.prod_id AND ps2.prod_id != ps3.prod_id AND ps1.prod_id!= ps3.prod_id
        GROUP BY 1,2,3
        ORDER BY trans_count DESC
        LIMIT 1;
'''

show(db='clothing_company', query=q)

,product_1,product_2,product_3,trans_count
0,5d267b,c8d436,9ec847,352
